## 1. Datenimport und Initialisierung

In diesem Abschnitt werden die OpenJur-Urteilstexte aus dem Datenverzeichnis eingelesen und die technische Datenbasis für die nachfolgenden Verarbeitungsschritte geschaffen. Dazu werden die benötigten Bibliotheken importiert und die verfügbaren Textdateien identifiziert.


### 1.1 Import der benötigten Bibliotheken

Zu Beginn werden die für die weitere Verarbeitung erforderlichen Python-Bibliotheken importiert. Diese umfassen Funktionen für Dateizugriffe, reguläre Ausdrücke, Datenverarbeitung mit Pandas sowie den Export der Ergebnisse im JSON-Format.

In [50]:
#Import
import os
import re
import json
import pandas as pd

### 1.2 Einlesen der OpenJur-Urteilstexte 

In diesem Schritt werden alle identifizierten Urteilstexte aus dem Datenverzeichnis eingelesen. Jede Datei wird über den Dateinamen einer eindeutigen Fallkennung (`case_id`) zugeordnet. Die Texte bilden die Rohdatenbasis für die nachfolgenden Extraktions- und Filterprozesse. Der Datenpfad wird im Code parametriert (`DATA_DIR`), um eine reproduzierbare Ausführung zu gewährleisten.

In [51]:
# (.txt) Dateien einlesen
DATA_DIR = "../data/Gerichtsurteile_Openjur" 
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".txt")]

print("Pfad:", os.path.abspath(DATA_DIR))
print("Anzahl .txt:", len(files))
print("Erste 10 Dateien:", files[:10])


Pfad: c:\Users\humme\OneDrive\Dokumente\Uni Ulm\ds_law\backend\data\Gerichtsurteile_Openjur
Anzahl .txt: 2375
Erste 10 Dateien: ['2090187.txt', '2112111.txt', '2112115.txt', '2112117.txt', '2112118.txt', '2112119.txt', '2112121.txt', '2112123.txt', '2124977.txt', '2126821.txt']


---

## 2. Extraktion relevanter Urteilsbestandteile und Selektion der Landgerichtsurteile

In diesem Abschnitt werden die eingelesenen Urteilstexte weiterverarbeitet, um für die nachfolgende Analyse relevante Textbestandteile gezielt zu extrahieren. Hierzu zählen insbesondere ein begrenzter Kopfbereich zur Voranalyse sowie der Tenor als Kern der gerichtlichen Entscheidung. Die strukturierte Aufbereitung dieser Textsegmente bildet die Grundlage für Filter-, Klassifikations- und Extraktionsschritte in den folgenden Abschnitten.

### 2.1 Aufbau des DataFrames und Extraktion eines Kopfbereichs

Die eingelesenen Texte werden in einem DataFrame (`df`) gespeichert. Zusätzlich wird ein begrenzter Kopfbereich (`head`) aus den ersten Zeichen extrahiert, da strukturelle Metadaten wie Gerichtstyp, Entscheidungsart und Zitierzeilen typischerweise am Anfang des Dokuments auftreten. Dieser Kopfbereich dient als effizienter Suchraum für die spätere Identifikation von Landgerichtsurteilen.


In [52]:
rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    path = os.path.join(DATA_DIR, fn)
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    rows.append({"case_id": case_id, "text": text})

df = pd.DataFrame(rows)
print("Gesamt eingelesen:", len(df))


Gesamt eingelesen: 2375


In [53]:
HEAD_CHARS = 8000
df["head"] = df["text"].astype(str).str.slice(0, HEAD_CHARS)

print("Head-Länge (Beispiel):", len(df.loc[0, "head"]))


Head-Länge (Beispiel): 8000


### 2.2 Extraktion des Tenors

Der Tenor enthält die eigentliche gerichtliche Entscheidung und ist daher für die inhaltliche Bewertung besonders relevant. Mithilfe regulärer Ausdrücke wird der Textabschnitt zwischen der Überschrift „Tenor“ und den nachfolgenden Abschnitten (z. B. „Tatbestand“ oder „Gründe“) extrahiert und in einer separaten Spalte gespeichert.

In [54]:
def extract_tenor(text: str) -> str:
    if not isinstance(text, str):
        return ""

    m_start = re.search(r"\bTenor\b", text, flags=re.IGNORECASE)
    if not m_start:
        return ""

    start = m_start.end()

    # Begrenztes Suchfenster nach dem Tenor (robuster gegen Navigation)
    window = text[start:start + 20000]

    m_end = re.search(
        r"\b(Tatbestand|Gründe|Gruende|Entscheidungsgründe|Entscheidungsgruende)\b",
        window,
        flags=re.IGNORECASE
    )

    end = start + m_end.start() if m_end else min(len(text), start + 8000)
    return text[start:end].strip()
df["tenor"] = df["text"].apply(extract_tenor)
print("Tenor vorhanden:", (df["tenor"].str.len() > 0).sum(), "von", len(df))


Tenor vorhanden: 2362 von 2375


### 2.3 Identifikation von Landgerichtsurteilen (LG)

Die Selektion der Landgerichtsurteile erfolgt anhand einer OpenJur-spezifischen Zitierzeile im Kopfbereich (Regex: „Einfach“ gefolgt von „LG“). Auf dieser Grundlage wird eine boolesche Variable erzeugt und der Teilkorpus df_lg gebildet.

In [55]:
# Wir suchen nach der Zeile, die mit "Einfach" beginnt, gefolgt von "LG"
# Der Regex r"Einfach\s*\n\s*LG" stellt sicher, dass LG direkt darunter steht
pattern_zitierung_lg = r"Einfach\s*\n\s*LG"

# Wir wenden das auf die Spalte an, die den Kopftext enthält
df["is_landgericht"] = df["head"].str.contains(pattern_zitierung_lg, regex=True, na=False)

# Jetzt erstellen wir den sauberen Dataframe
df_lg = df[df["is_landgericht"] == True].copy()

print("-" * 40)
print(f"✅ Echte LG-Urteile (über Zitierzeile): {len(df_lg)}")
print("-" * 40)

----------------------------------------
✅ Echte LG-Urteile (über Zitierzeile): 1189
----------------------------------------


### 2.4 Segmentierung der Urteile in juristische Abschnitte
Für die spätere Extraktion werden die Urteile in juristisch sinnvolle Teile zerlegt: Rubrum, Tenor, Tatbestand und Entscheidungsgründe. Dadurch kann das Modell gezielt relevante Passagen verarbeiten.
Die Segmentierung dient dazu, spätere Analysen gezielt auf entscheidungsrelevante Abschnitte (insb. Tenor und Entscheidungsgründe) zu fokussieren.


In [56]:
def split_judgment(text):
    """
    Teilt ein Urteil in Rubrum, Tenor, Tatbestand und Entscheidungsgründe auf.
    """
    segments = {
        "rubrum": "",
        "tenor": "",
        "tatbestand": "",
        "entscheidungsgruende": ""
    }
    
    # Muster für die Abschnittsüberschriften
    # Das Rubrum ist alles vor dem Tenor
    m_tenor = re.search(r"\bTenor\b", text, re.IGNORECASE)
    m_tatbestand = re.search(r"\bTatbestand\b", text, re.IGNORECASE)
    m_gruende = re.search(r"\b(Entscheidungsgründe|Entscheidungsgruende|Gründe|Gruende)\b", text, re.IGNORECASE)
    
    if m_tenor:
        segments["rubrum"] = text[:m_tenor.start()].strip()
        
        # Tenor bis Tatbestand
        if m_tatbestand:
            segments["tenor"] = text[m_tenor.end():m_tatbestand.start()].strip()
            
            # Tatbestand bis Gründe
            if m_gruende:
                segments["tatbestand"] = text[m_tatbestand.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tatbestand"] = text[m_tatbestand.end():].strip()
        else:
            # Falls kein Tatbestand gefunden wird, Tenor bis zum Ende oder Gründen
            if m_gruende:
                segments["tenor"] = text[m_tenor.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tenor"] = text[m_tenor.end():].strip()
                
    return segments

# Beispielanwendung auf den Dataframe
df_lg['segments'] = df_lg['text'].apply(split_judgment)

## 3 Prompt-Generierung und Pilotierung der LLM-Extraktion (Gemini Batch)
Um API- und Token-Limits zu berücksichtigen, werden OpenJur-spezifische Navigationselemente aus dem Rubrum entfernt und alle Abschnitte in ihrer Länge begrenzt. Auf Basis dieser vorverarbeiteten Textsegmente wird ein standardisierter Prompt generiert, der die Extraktion der abgestimmten Variablen im JSON-Format steuert.
Die segmentweise Längenbegrenzung dient der Einhaltung von Token-Limits sowie der Reduktion von Kosten und Laufzeit, ohne entscheidungsrelevante Passagen (insb. Tenor und Entscheidungsgründe) zu verlieren.


### 3.1 Aufbereitung der Segmente und Definition des Extraktions-Prompts

Der Prompt wurde so konzipiert, dass er neben technischen Features (Motor, Kilometer) gezielt die Anforderungen der Aufgabenstellung erfüllt. Kernaspekte sind die Identifikation des Gerichtstyps sowie die Differenzierung der Zielvariable in Schadensersatz, Klageabweisung und prozessuale Sonderfälle („Sonstige“). Durch explizite Anweisungen zum Ausschluss von Zinsen und zur Erkennung von Streitwertbeschlüssen wird eine hohe Datenqualität für das anschließende Machine Learning sichergestellt.

In [57]:
def clean_rubrum(rubrum: str) -> str:
    if not isinstance(rubrum, str):
        return ""

    blacklist = [
        "rechtsprechung", "aktuell", "trending", "filter",
        "über openjur", "spenden", "api", "hilfe",
        "startseite", "bundesland", "gerichtsbarkeit",
        "impressum", "datenschutz", "nutzungsbedingungen",
        "fachzeitschriften", "suchen", "changelog", "einfach",
        "json", "bibtex", "ris"
    ]

    lines = []
    for line in rubrum.splitlines():
        l = line.strip().lower()
        if not l:
            continue
        if any(b in l for b in blacklist):
            continue
        lines.append(line.strip())

    return "\n".join(lines[:5])   

def slim_segments(segments):
    return {
        "rubrum": clean_rubrum(segments.get("rubrum") or "")[:2500],
        "tenor": (segments.get("tenor") or "")[:4000],
        "tatbestand": (segments.get("tatbestand") or "")[:3500],
        "entscheidungsgruende": (segments.get("entscheidungsgruende") or "")[:7000],
    }
def get_gemini_prompt(segments):
    """
    Erstellt den finalen Prompt basierend auf den Urteilssegmenten.
    """
    s = slim_segments(segments)

    prompt = f"""
Analysiere die folgenden Abschnitte eines Gerichtsurteils zum Dieselskandal und extrahiere die Variablen präzise als JSON-Liste. 

### URTEILS-BESTANDTEILE:
RUBRUM (Kopfbereich mit Gericht & Datum): 
{s['rubrum']}

TENOR (Ergebnis): 
{s['tenor']}

TATBESTAND (Sachverhalt): 
{s['tatbestand']}

ENTSCHEIDUNGSGRÜNDE (Rechtliche Würdigung): 
{s['entscheidungsgruende']}

### EXTRAKTIONS-AUFGABE:
Extrahiere folgende Variablen (bei Nichtfinden 'null' angeben):

WICHTIG (Validierung & Datenqualität):
1) **Gerichtstyp** muss explizit angegeben werden (z.B. "Landgericht", "Oberlandesgericht", "Amtsgericht").  
2) **Sonstige-Kategorie (prozessuale Dokumente):** Falls das Dokument **keine materielle Entscheidung über einen Schadensersatzanspruch** enthält (z.B. nur Streitwertfestsetzung/-beschluss, Prozesskostenhilfe/PKH, Kostenentscheidung ohne Sachentscheidung, Ablehnungsgesuch/Befangenheit, rein prozessualer Beschluss), dann setze zwingend:
   - LABEL_Anspruch_Schadensersatz = false
   - LABEL_Schadensersatzhoehe_Betrag = null
   - LABEL_Schadensersatzhoehe_Range = "Sonstige"
3) **Betrag ohne Zinsen:** LABEL_Schadensersatzhoehe_Betrag ist **ohne Zinsen/Verzugszinsen/Nebenforderungen** anzugeben.

1. **Input-Variablen (Features):**
   - Dieselmotor_Typ: (Beispiel: "EA 189", "EA 288")
   - Art_Abschalteinrichtung: (Beispiel: "Umschaltlogik", "Thermofenster")
   - KBA_Rueckruf: (Boolean: true/false - Beispiel: true)
   - Fahrzeugstatus: ("Neuwagen" oder "Gebrauchtwagen")
   - Fahrzeugmodell_Baureihe: (Beispiel: "VW Golf 2.0 TDI")
   - Update_Status: (Boolean: true/false/null - Beispiel: false)
   - Kilometerstand_Kauf: (Integer - Beispiel: 15200)
   - Kilometerstand_Klageerhebung: (Integer - Beispiel: 45000)
   - Erwartete_Gesamtlaufleistung: (Integer - Beispiel: 250000)
   - Kaufdatum: (Date YYYY-MM-DD - Beispiel: 2014-05-12)
   - Uebergabedatum: (Date YYYY-MM-DD - Beispiel: 2014-05-20)
   - Datum_Klageerhebung: (Date YYYY-MM-DD - Beispiel: 2018-11-03)
   - Beklagten_Typ: ("Händler" oder "Hersteller")
   - Datum_Urteil: (Date YYYY-MM-DD - Beispiel: 2019-12-17)
   - Kaufpreis: (Float in EUR - Beispiel: 25900.00)
   - Nacherfuellungsverlangen_Fristsetzung: ("Ja", "Nein", "Entbehrlich")
   - Klageziel: ("Rückabwicklung", "Minderung", "Schadensersatz")
   - Rechtsgrundlage: (Beispiel: "§ 826 BGB", "§ 437 BGB")

2. **Zielvariablen (Labels):**
   - LABEL_Anspruch_Schadensersatz (Boolean: true/false - Beispiel: true)
   - LABEL_Schadensersatzhoehe_Betrag (Float in EUR - Beispiel: 18450.50)
   - LABEL_Schadensersatzhoehe_Range (Beispiel: "< 5000", "5000-10000", "10000-15000", "15000-20000", "20000-25000", "> 25000", "Klage abgewiesen")

### AUSGABEFORMAT:
Antworte NUR mit einem validen JSON-Objekt in einer Liste:
[{{
  "case_id": "...",
  "Gerichtstyp": null,
  "Dieselmotor_Typ": null,
  "Art_Abschalteinrichtung": null,
  "KBA_Rueckruf": null,
  "Fahrzeugstatus": null,
  "Fahrzeugmodell_Baureihe": null,
  "Update_Status": null,
  "Kilometerstand_Kauf": null,
  "Kilometerstand_Klageerhebung": null,
  "Erwartete_Gesamtlaufleistung": null,
  "Kaufdatum": null,
  "Uebergabedatum": null,
  "Datum_Klageerhebung": null,
  "Nachweis_Aufklaerung": null,
  "Beklagten_Typ": null,
  "Datum_Urteil": null,
  "Kaufpreis": null,
  "Nacherfuellungsverlangen_Fristsetzung": null,
  "Klageziel": null,
  "Rechtsgrundlage": null,
  "LABEL_Anspruch_Schadensersatz": null,
  "LABEL_Schadensersatzhoehe_Betrag": null,
  "LABEL_Schadensersatzhoehe_Range": null
}}]
""".strip()

    return prompt


### 3.2 Erstellung eines Pilot-Inputs im JSONL-Format
Zur technischen Validierung der Analysepipeline wird ein Pilotdatensatz erzeugt, der eine begrenzte Anzahl von Landgerichtsurteilen umfasst. Für jedes ausgewählte Urteil werden die zuvor definierten Textsegmente extrahiert, zu einem standardisierten Analyse-Prompt zusammengeführt und im JSONL-Format gespeichert. Diese Pilotdatei dient als Testeingabe für die nachgelagerte Verarbeitung über die Gemini-API, bevor eine Skalierung auf den vollständigen Datensatz erfolgt.
Der Pilot dient ausschließlich der technischen Validierung der Prompt-Struktur und der Batch-Pipeline und ist nicht für eine inhaltliche Evaluation der Extraktionsergebnisse vorgesehen.

In [58]:
PILOT_N = 10
pilot_path = "gemini_batch_input_pilot_10.jsonl"

with open(pilot_path, "w", encoding="utf-8") as f:
    for _, row in df_lg.head(PILOT_N).iterrows():
        segments = row["segments"]
        full_prompt = get_gemini_prompt(segments)

        payload = {
            "custom_id": f"case_{row['case_id']}",
            "contents": [{
                "role": "user",
                "parts": [{"text": full_prompt}]
            }]
        }
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print("✅ Pilot erstellt (aus Trainingssplit):", pilot_path)


✅ Pilot erstellt (aus Trainingssplit): gemini_batch_input_pilot_10.jsonl


Der Code dient der inhaltlichen und technischen Validierung der erzeugten Pilotdatei. Hierzu wird der erste Eintrag der JSONL-Datei geladen und exemplarisch ausgegeben, um Struktur, Inhalt und Länge des generierten Analyse-Prompts zu überprüfen

In [59]:
with open("gemini_batch_input_pilot_10.jsonl", "r", encoding="utf-8") as f:
    first = json.loads(f.readline())

print(first["custom_id"])
print(first["contents"][0]["parts"][0]["text"][:800])
print("Prompt-Länge:", len(first["contents"][0]["parts"][0]["text"]))


case_2090187
Analysiere die folgenden Abschnitte eines Gerichtsurteils zum Dieselskandal und extrahiere die Variablen präzise als JSON-Liste. 

### URTEILS-BESTANDTEILE:
RUBRUM (Kopfbereich mit Gericht & Datum): 
Rechtsgebiet
Gericht
Informationen
Urschrift des Grundgesetzes
Abkürzungen

TENOR (Ergebnis): 
I. Die Klage wird abgewiesen.II. Der Kläger hat die Kosten des Rechtsstreits zu tragen.III. Das Urteil ist gegen Sicherheitsleistung in Höhe des 1,1-fachen des zu vollstreckenden Betrags vorläufig vollstreckbar.IV. Der Streitwert wird auf 31.234,00 € festgesetzt.

TATBESTAND (Sachverhalt): 
Der Kläger begehrt Lieferung eines mangelfreien Pkw.Der Kläger erwarb von der Beklagten im Jahr 2014 einen Neuwagen VW Passat 2,0 l TDI für 31.234,00 €. Der Pkw ist von dem "VW-Abgasskandal" betroffen. Der Kläger 
Prompt-Länge: 10554


### 3.3 Upload und Start eines Pilot-Batch-Jobs
In diesem Schritt wird die zuvor erzeugte Pilot-JSONL-Datei als Eingabe für die Gemini-API hochgeladen. Die Datei enthält strukturierte Analyseanfragen für mehrere Urteile und wird auf den Servern bereitgestellt, sodass sie anschließend im Rahmen einer Batch- oder sequenziellen Verarbeitung vom Sprachmodell verarbeitet werden kann. Der Upload erzeugt eine referenzierbare Eingabedatei, die anschließend einem eindeutig benannten Batch-Job zugewiesen wird und damit eine reproduzierbare Verarbeitung durch das Sprachmodell ermöglicht.


Initialisierung des API-Clients

In [60]:
from google import genai
import os
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise RuntimeError("GEMINI_API_KEY ist nicht gesetzt")

client = genai.Client(api_key=api_key)
print("Client initialisiert")

Client initialisiert


Upload der JSONL-Datei

In [61]:
uploaded = client.files.upload(
    file="gemini_batch_input_pilot_10.jsonl",
    config={
        "display_name": "diesel-lg-pilot-10",
        "mime_type": "application/jsonl"
    }
)
print("Upload:", uploaded.name)


Upload: files/qfh3004t59cj


In [62]:
job = client.batches.create(
    model="models/gemini-2.5-flash",
    src=uploaded.name,
    config={"display_name": "diesel-lg-pilot-10"}
)

print("Batch gestartet:", job.name)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}

## 4 Verarbeitung der Modellantworten und Erstellung des Extraktions-Datensatzes

In diesem Abschnitt werden die Batch-Ausgaben der Gemini-API eingelesen, validiert und in ein tabellarisches Format überführt. Zum Zeitpunkt der aktuellen Notebook-Version liegt lediglich der Pilot-Workflow vor; der Code zur Verarbeitung des vollständigen Batch-Outputs wird nach Abschluss des Batch-Jobs ergänzt.

(gemini_batch_input_NUR_LG.jsonl)

In [ ]:
# Zentrale Konfiguration für den Datenimport
# Wenn wir den echten Batch machen, müssen wir den Dateinamen hier anpassen
BATCH_OUTPUT_FILENAME = "gemini_batch_output_pilot_10.jsonl" 
DATA_INPUT_DIR = "./" # Verzeichnis, in dem die Batch-Datei liegt

print(f"System bereit für Import von: {BATCH_OUTPUT_FILENAME}")

System bereit für Import von: gemini_batch_output_pilot_10.jsonl


### 4.1 Download/Export der Batch-Ausgabedatei (JSONL) (Platzhalter)

Nach der in Abschnitt 2 beschriebenen Aufbereitung der Urteilstexte liegt der vollständige Analyse-Datensatz in Form einer strukturierten JSONL-Datei vor. Diese Datei dient in diesem Schritt als Eingabe für die automatisierte Verarbeitung durch ein großes Sprachmodell.

Die JSONL-Datei wird zunächst in das Batch-System hochgeladen. Anschließend wird ein Batch-Verarbeitungsjob gestartet, der die hochgeladene Datei als Eingabequelle verwendet. Für jedes enthaltene Dokument erzeugt das Modell eine strukturierte Antwort gemäß den im Prompt definierten Extraktionsvorgaben.

Als Ergebnis des Batch-Jobs stellt die API eine Ausgabedatei bereit, die die Modellantworten zu allen verarbeiteten Urteilen enthält. Diese Ausgabedatei liegt ebenfalls im JSONL-Format vor und bildet die Grundlage für die weitere Aufbereitung und Auswertung der Ergebnisse.


In [ ]:
import os

# 1. Konfiguration (Pfade müssen zu deiner Umgebung passen)
# BATCH_OUTPUT_FILENAME ist der Name der Datei, die du von Gemini erhalten hast
full_batch_path = os.path.join(DATA_INPUT_DIR, BATCH_OUTPUT_FILENAME)

# 2. Existenzprüfung: Sicherstellen, dass die Datei vorhanden ist
if os.path.exists(full_batch_path):
    # 3. Einlesen der Batch-Ausgabedatei (JSONL-Format)
    # Wir lesen zeilenweise ein, um den Speicher bei vielen Urteilen zu schonen
    with open(full_batch_path, "r", encoding="utf-8") as f:
        # Jede Zeile der JSONL ist ein String, der später in 4.2 geparst wird
        raw_batch_lines = [line.strip() for line in f if line.strip()]
    
    print(f"✅ Datei erfolgreich lokalisiert: {full_batch_path}")
    print(f"📊 Anzahl der geladenen KI-Antworten: {len(raw_batch_lines)}")
    
    # Kurzer Blick auf die Rohdaten zur Kontrolle
    if len(raw_batch_lines) > 0:
        print("\nErste Zeile Rohdaten (Vorschau):")
        print(raw_batch_lines[0][:150] + "...")
else:
    print(f"❌ FEHLER: Datei nicht gefunden unter {full_batch_path}")
    print("Bitte prüfe den Dateinamen oder lade die .jsonl Datei in das Verzeichnis hoch.")
    raw_batch_lines = []

❌ FEHLER: Datei nicht gefunden unter ./gemini_batch_output_pilot_10.jsonl
Bitte prüfe den Dateinamen oder lade die .jsonl Datei in das Verzeichnis hoch.


In [ ]:
print("raw_batch_lines:", len(raw_batch_lines))


raw_batch_lines: 0


### 4.2 Parsing, Validierung und Tabellierung der Extraktionen (Platzhalter)

Die im vorherigen Schritt erzeugte Ausgabedatei des Batch-Jobs liegt zunächst als Rohdaten im JSONL-Format vor. Jede Zeile dieser Datei enthält die strukturierte Modellantwort zu einem einzelnen Landgerichtsurteil.

Diese Rohdaten werden lokal gespeichert und anschließend in ein tabellarisches Format überführt. Hierzu werden die relevanten Felder aus den JSON-Strukturen extrahiert und in einer einheitlichen Datenstruktur zusammengeführt, beispielsweise in Form einer CSV-Datei. 
Der so erzeugte Datensatz bildet die Grundlage für die weitere statistische Auswertung und Analyse in den folgenden Abschnitten.

In [ ]:
import json
import re
import pandas as pd

def extract_json_from_llm(text: str):
    """
    Extrahiert den ersten JSON-Block aus einer LLM-Antwort und parsed ihn.
    Erwartet typischerweise eine Liste mit einem Objekt: [ { ... } ]
    """
    if not isinstance(text, str):
        raise ValueError("LLM-Output ist kein String")

    # Entfernt ```json ... ``` falls vorhanden
    text = re.sub(r"^```json\s*|\s*```$", "", text.strip(), flags=re.MULTILINE)

    # Falls außenrum Text steht: ersten JSON-Block (Liste oder Objekt) extrahieren
    m = re.search(r"(\[\s*\{.*?\}\s*\]|\{.*?\})", text, flags=re.DOTALL)
    if not m:
        raise ValueError("Kein JSON-Block in LLM-Output gefunden")

    return json.loads(m.group(1))

def to_num(val):
    """
    Konvertiert textuelle Beträge (DE/EN Formate) robust in numerische Werte.
    Beispiele: '25.900,50 EUR', '25,900.50', '25900,50'.
    """
    if val is None:
        return None

    s = str(val).strip().lower()
    if s in {"null", "nan", "none", ""}:
        return None

    # Entfernt Währungszeichen und Textreste, behält nur Ziffern und Trenner
    s = re.sub(r"[^\d.,]", "", s)

    # Behandlung von Mischformaten (Deutsch vs. Englisch)
    if "," in s and "." in s:
        # Deutsch: 25.900,50 -> 25900.50
        s = s.replace(".", "").replace(",", ".") if s.find(".") < s.find(",") else s.replace(",", "")
    elif "," in s:
        # Reines Kommaformat -> Dezimalpunkt
        s = s.replace(",", ".")

    try:
        return float(s)
    except ValueError:
        return None

# Container für erfolgreiche Extraktionen und Parsing-Fehler
rows, errs = [], []

# Iteration über jede Zeile der Batch-Ausgabedatei (JSONL)
for line in raw_batch_lines:
    case_id = None
    try:
        # 1) Parsen der JSONL-Zeile (Batch-Wrapper)
        b = json.loads(line)

        # 2) Extraktion der eindeutigen Fall-ID
        case_id = b.get("custom_id", "").replace("case_", "").strip()

        # 3) Extraktion des reinen Modelltexts aus dem Batch-Wrapper
        raw = b["response"]["body"]["choices"][0]["message"]["content"]

        # 4) JSON-Extraktion aus der Modellantwort
        d = extract_json_from_llm(raw)

        # Normalisierung: Prompt liefert meist eine Liste mit genau einem Objekt
        d = d[0] if isinstance(d, list) else d
        if not isinstance(d, dict):
            raise ValueError("Extrahiertes JSON ist kein Objekt (dict)")

        # 5) Sicherung der Fall-ID für spätere Zusammenführung
        d["case_id"] = case_id

        # 6) Feature Engineering: numerische Beträge
        d["Schadensersatz_Betrag_num"] = to_num(d.get("LABEL_Schadensersatzhoehe_Betrag"))
        d["Kaufpreis_num"] = to_num(d.get("Kaufpreis"))

        # 7) Konsistenzregeln & Zielklassenbildung (robust)
        is_sonstige = str(d.get("LABEL_Schadensersatzhoehe_Range", "")).strip().lower() == "sonstige"
        if is_sonstige:
            d.update({
                "target_label": "Sonstige",
                "LABEL_Anspruch_Schadensersatz": False,
                "LABEL_Schadensersatzhoehe_Betrag": None,
                "Schadensersatz_Betrag_num": None
            })
        else:
            val = d.get("LABEL_Anspruch_Schadensersatz")
            is_true = (val is True) or (isinstance(val, str) and val.strip().lower() == "true")

            if is_true:
                d["target_label"] = "Schadensersatz"
            else:
                d["target_label"] = "Abgewiesen"

        # Erfolgreich verarbeiteter Datensatz
        rows.append(d)

    except Exception as e:
        # Fehlerhafte Batch-Zeilen werden dokumentiert, nicht verworfen
        errs.append({"case_id": case_id, "error": str(e)})

# 8) Überführung der bereinigten Extraktionen in ein tabellarisches Format
df_extracted = pd.DataFrame(rows)

# Wichtig: Damit 4.3 nicht an assert(case_id) scheitert, auch wenn rows leer ist
if df_extracted.empty:
    df_extracted = pd.DataFrame(columns=["case_id"])

# Kurzer Qualitätsreport
print(f"✅ 4.2 abgeschlossen: {len(df_extracted)} Datensätze extrahiert")
print(f"⚠️ Parsing-Fehler: {len(errs)}")

# Optional: Fehlerliste kurz anzeigen
if len(errs) > 0:
    print("Beispiel-Fehler:", errs[0])


✅ 4.2 abgeschlossen: 0 Datensätze extrahiert
⚠️ Parsing-Fehler: 0


### 4.3 Zusammenführung mit Metadaten und Speicherung (CSV/Parquet) (Platzhalter)

In diesem finalen Schritt der Datenextraktion werden die bereinigten Modellantworten (df_clean_labels) mit den ursprünglichen Metadaten und Urteilstexten der Landgerichte (df_lg) zusammengeführt. Die Verknüpfung erfolgt über die eindeutige case_id, um eine konsistente Zuordnung zwischen den technischen Features (z. B. Motortyp, Kilometerstand) und den Zielvariablen (Schadensersatzhöhe, Anspruchsstatus) zu gewährleisten.
Der resultierende Gesamtdatensatz wird in zwei Formaten exportiert:
* CSV-Format: Zur einfachen manuellen Überprüfung der Extraktionsergebnisse in Tabellenkalkulationsprogrammen.
* Parquet-Format: Zur effizienten Weiterverarbeitung in der Machine-Learning-Phase (Kapitel 5), da dieses Format Datentypen (z. B. numerische Beträge ohne Zinsen) verlustfrei speichert.
Damit ist die Datenbasis für die nachfolgende semantische Analyse und Modellierung vollständig vorbereitet.---

In [ ]:
# 1) Sicherstellen, dass case_id in beiden DataFrames existiert (läuft erst mit echtem Batch)
assert "case_id" in df_lg.columns, "df_lg enthält keine case_id"
assert "case_id" in df_extracted.columns, "df_extracted enthält keine case_id"

# 2) Zusammenführung: Urteilstexte + Metadaten + extrahierte Labels
df_dataset = pd.merge(
    df_lg,
    df_extracted,
    on="case_id",
    how="inner"
)

print(f"✅ Merge abgeschlossen: {df_dataset.shape[0]} Urteile im Gesamtdatensatz")

# 3) Sanity-Checks
print(df_dataset["target_label"].value_counts(dropna=False))

# 4) Export
OUTPUT_BASENAME = "lg_diesel_urteile_final"
df_dataset.to_csv(f"{OUTPUT_BASENAME}.csv", index=False, encoding="utf-8")
df_dataset.to_parquet(f"{OUTPUT_BASENAME}.parquet", index=False)

print("Export abgeschlossen: CSV & Parquet erstellt")


✅ Merge abgeschlossen: 0 Urteile im Gesamtdatensatz


KeyError: 'target_label'

### 4.4 Datenaufteilung und Validierungskonzept

Der Datensatz wird auf Fall-Ebene (`case_id`) in einen Trainings- (80 %) und einen Testdatensatz (20 %) aufgeteilt.  
Der Trainingsdatensatz wird anschließend mittels **5-facher stratified Cross-Validation** für die Modellselektion und Hyperparameter-Optimierung genutzt.

Dieses Vorgehen erlaubt eine effiziente Nutzung der verfügbaren, kostenintensiv extrahierten Labels, während der Testdatensatz vollständig unberührt bleibt und ausschließlich zur finalen Evaluation der Modellleistung dient.

In [ ]:
# Wir trennen die Daten strikt in Training (80%) und Test (20%).

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

# 1. Features auswählen (NUR Input-Daten, keine Ergebnisse!)
# Wir nehmen nur den Kaufpreis als strukturiertes Feature.
X_structured = df_dataset[['Kaufpreis_num']].fillna(0) 

# Wir nehmen den rohen Text (oder text_for_embedding) für den Split
X_text = df_dataset['text'] 

# Ziel-Variable
y = df_dataset['LABEL_Anspruch_Schadensersatz'].astype(int)

# 2. Der Split
X_train_text, X_test_text, X_train_struct, X_test_struct, y_train, y_test = train_test_split(
    X_text, X_structured, y, 
    test_size=0.20, 
    random_state=42, 
    stratify=y  # Wichtig für gleiche Klassenverteilung!
)

# 3. DataFrames für Kapitel 5 erstellen (macht das Preprocessing einfacher)
df_lg_train = pd.DataFrame({'text': X_train_text, 'LABEL_Anspruch_Schadensersatz': y_train}).join(X_train_struct)
df_lg_test = pd.DataFrame({'text': X_test_text, 'LABEL_Anspruch_Schadensersatz': y_test}).join(X_test_struct)

# 4. CV-Objekt für Abschnitt 6 bereitstellen
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"✅ Split abgeschlossen: Training ({len(df_lg_train)}), Test ({len(df_lg_test)})")

## 5. Datenaufbereitung für maschinelles Lernen

In diesem Abschnitt werden die Urteilstexte für die nachgelagerte prädiktive Modellierung aufbereitet. Hierzu erfolgt zunächst eine juristisch angepasste Textvorverarbeitung und die Ableitung numerischer Textrepräsentationen. Die für die supervised Lernphase erforderlichen Zielvariablen werden im Rahmen der LLM-basierten Extraktion (Abschnitt 4) erzeugt und anschließend mit den Textmerkmalen zusammengeführt (Abschnitt 5.4).
Ziel der Datenaufbereitung ist es, die extrahierten Merkmale in eine konsistente, auswertbare Form zu überführen, fehlende oder uneinheitliche Angaben zu behandeln und die Zielvariablen für die spätere Analyse eindeutig zu definieren.

### 5.1 Juristische Textvorverarbeitung

Wir wenden die spezialisierte Reinigung auf den Trainings- und Testdatensatz an.


In [ ]:
import pandas as pd
import re
import spacy
import json
from tqdm import tqdm

# 1. Setup: Spezialisiertes deutsches Sprachmodell laden
try:
    # Wir deaktivieren unnötige Komponenten (ner, parser), um die Verarbeitung zu beschleunigen
    nlp = spacy.load("de_core_news_lg", disable=["ner", "parser"])
except Exception:
    print("Bitte installiere das spacy Modell: python -m spacy download de_core_news_lg")

# --- 2. JURISTISCHE TEXTVORVERARBEITUNG ---
def legal_preprocess(text):
    """
    Bereitet juristische Texte auf, indem Rauschen entfernt wird, 
    während rechtlich relevante Zahlen und Kontexte geschützt werden.
    """
    if not isinstance(text, str) or not text:
        return ""

    # START DES URTEILS FINDEN (Rauschschnitt Anfang)
    start_keywords = ["tenor", "entscheidungsgründe", "tatbestand", "urteil", "beschluss", "endurteil"]
    text_lower_start = text.lower()
    found_positions = [text_lower_start.find(kw) for kw in start_keywords if text_lower_start.find(kw) != -1]
    if found_positions:
        text = text[min(found_positions):]

    # ENDE DES URTEILS FINDEN (Rauschschnitt Ende)
    end_keywords = ["impressum", "nutzungsbedingungen", "nach oben", "datenschutz"]
    text_lower_end = text.lower()
    for ekw in end_keywords:
        e_pos = text_lower_end.find(ekw)
        if e_pos != -1:
            text = text[:e_pos]
            break

    # 1. Bereinigung von Rauschen (HTML-Tags, Sonderzeichen)
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Schutz von Zahlen & Paragraphen (Platzhalter)
    text = re.sub(r'\d{1,3}(?:\.\d{3})(?:,\d+)?\s(?:EUR|€|Euro)', ' PLATZHALTER_BETRAG ', text)
    text = re.sub(r'§+\s*\d+[a-z]?\s*(?:\w+)?', ' PLATZHALTER_PARAGRAPH ', text)
    text = re.sub(r'\b(19|20)\d{2}\b', ' PLATZHALTER_JAHR ', text)

    # 3. Kleinschreibung
    text = text.lower()

    # 4. Tokenisierung und Lemmatisierung mit SpaCy
    doc = nlp(text)
    
    # 5. Kontextsensitive Stoppwort-Entfernung
    protected_negations = {"nicht", "kein", "ohne", "gegen", "trotz"}
    custom_stop_words = nlp.Defaults.stop_words - protected_negations
    
    tokens = [
        token.lemma_ for token in doc 
        if token.lemma_ not in custom_stop_words 
        and not token.is_punct 
        and not token.is_space
        and len(token.text) > 1
    ]
    
    return " ".join(tokens)

# --- 2.5 HILFSFUNKTIONEN ---
def get_llm_text(r: dict) -> str:
    if "text" in r: return r["text"]
    if "response" in r: return r["response"]["body"]["choices"][0]["message"]["content"]
    raise KeyError("Unbekanntes Ergebnisformat.")

def parse_llm_json(text: str) -> dict:
    text = re.sub(r"^json\s*|\s*$", "", text.strip(), flags=re.MULTILINE)
    m = re.search(r"(\{.*\})", text, flags=re.DOTALL)
    if m: text = m.group(1)
    return json.loads(text)

# --- 3. MERGING DER DATEN ---
def merge_and_finalize(judgment_df, batch_results_file):
    """
    Führt die Urteilstexte mit den Gemini-Extraktionen zusammen.
    Hier wird legal_preprocess NICHT mehr automatisch auf alles angewendet.
    """
    with open(batch_results_file, 'r', encoding='utf-8') as f:
        results = [json.loads(line) for line in f]
    
    extracted_rows = []
    for r in results:
        try:
            case_id = r['custom_id'].replace('case_', '')
            content = parse_llm_json(get_llm_text(r))
            content['case_id'] = case_id
            extracted_rows.append(content)
        except Exception:
            continue
            
    df_extracted = pd.DataFrame(extracted_rows)
    return pd.merge(judgment_df, df_extracted, on='case_id', how='inner')

# --- AUSFÜHRUNG NUR FÜR TRAININGSDATEN ---
# Wir aktivieren tqdm für den Fortschrittsbalken
tqdm.pandas()

print("Starte Vorverarbeitung der Trainingsdaten...")
# Wir wenden die Reinigung nur auf df_lg_train an (erstellt in Schritt 4.4)
df_lg_train['cleaned_text'] = df_lg_train['text'].progress_apply(legal_preprocess)

print(f"✅ Preprocessing abgeschlossen. Datensätze im Training: {len(df_lg_train)}")

Starte Vorverarbeitung der Trainingsdaten...


  0%|          | 0/951 [00:00<?, ?it/s]

AttributeError: 'Series' object has no attribute '_is_builtin_func'

In dieser Zelle extrahieren wir die relevanten Abschnitte (Tenor + Gründe) und wenden das Preprocessing ausschließlich auf die Trainingsdaten an.


In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Wir arbeiten NUR mit dem Trainingsdatensatz aus Abschnitt 4.4
# (Sollte df_lg_train noch nicht existieren, stelle sicher, dass 4.4 ausgeführt wurde)
df_train_work = df_lg_train.copy()

# 1) Sicherstellen, dass Segmente existieren
if "segments" not in df_train_work.columns:
    print("Extrahiere Segmente für Trainingsdaten...")
    df_train_work["segments"] = df_train_work["text"].apply(split_judgment)

# 2) Textbasis für ML definieren: Fokus auf TENOR + ENTSCHEIDUNGSGRÜNDE
# Diese Abschnitte enthalten die juristische Essenz für Word2Vec
def build_text_for_embedding(s):
    if not isinstance(s, dict):
        return ""
    # Wir kombinieren Tenor und Gründe, da hier die Begründung für Schadensersatz steht
    return (s.get("tenor") or "") + "\n" + (s.get("entscheidungsgruende") or "")

df_train_work["text_for_embedding"] = df_train_work["segments"].apply(build_text_for_embedding)

# 3) Länge begrenzen (Token-/Laufzeitkontrolle)
# 12.000 Zeichen sind meist ausreichend, um die Kernargumente zu erfassen
MAX_CHARS = 12000
df_train_work["text_for_embedding"] = (
    df_train_work["text_for_embedding"]
    .astype(str)
    .str.slice(0, MAX_CHARS)
)

# 4) Juristisches Preprocessing (nur Trainingsdaten)
# Hier wird die in der vorherigen Zelle definierte Funktion 'legal_preprocess' genutzt
print("Starte juristische Vorverarbeitung der Trainingsdaten...")
df_train_work["cleaned_text"] = df_train_work["text_for_embedding"].progress_apply(legal_preprocess)

# 5) Zurückschreiben in den Haupt-Trainings-DataFrame
df_lg_train = df_train_work

# 6) Sanity-Checks
print("-" * 30)
print(f"✅ Training-Datensatz bereit: {df_lg_train.shape[0]} Urteile")
print(f"📊 Davon erfolgreich bereinigt: {(df_lg_train['cleaned_text'].str.len() > 0).sum()}")

print("\nVorschau der bereinigten Trainings-Tokens:")
print(df_lg_train["cleaned_text"].iloc[0][:300] + "...")

### 5.2 Text-Vektorisierung mittels Word2Vec
Das Modell lernt semantische Relationen ausschließlich aus dem Trainingskorpus.

In [ ]:
from gensim.models import Word2Vec

# 1. Sicherstellen, dass wir den richtigen DataFrame nutzen
df_train = df_lg_train 

# 2. Tokenisierung: Liste von Wortlisten erstellen
# Wir nutzen .dropna(), falls durch das Preprocessing leere Felder entstanden sind
train_sentences = [str(text).split() for text in df_train["cleaned_text"] if text]

# 3. Training des Word2Vec-Modells
# sg=1 (Skip-Gram) ist top für juristische Nuancen
print("Training des Word2Vec-Modells auf den Trainingsdaten...")
w2v_model = Word2Vec(
    sentences=train_sentences,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    sg=1,
    seed=42
)

# 4. Vorbereitung der Zielvariable (y)
# Wir stellen sicher, dass y_train exakt zu den Zeilen in df_train passt
y_train = df_train["LABEL_Anspruch_Schadensersatz"].astype(int)

print(f"✅ Word2Vec-Modell trainiert. Vokabulargröße: {len(w2v_model.wv)} Wörter.")
print(f"✅ Zielvariable y_train erstellt (N={len(y_train)}).")

### 5.3 Aufbau des Analyse-Datensatzes
Nach der Vektorisierung führen wir die mathematischen Ergebnisse in einer strukturierten Feature-Matrix zusammen. Wir wandeln die Sparse-Matrix in einen übersichtlichen DataFrame um und verknüpfen jedes Urteil über die eindeutige case_id mit seinen Textmerkmalen. Diese Struktur ist essentiell, um im nächsten Schritt die durch das LLM extrahierten Zielvariablen (Schadensersatz oder Abweisung) präzise jeder Beobachtung zuordnen zu können. Damit stellen wir sicher, dass der Datensatz modellunabhängig konzipiert ist und eine solide Basis für die nachgelagerte prädiktive Modellierung bietet.

In [ ]:
import numpy as np

# 1. Hilfsfunktion zur Erstellung eines Dokument-Vektors
def get_doc_vector(doc, model):
    """ Erstellt einen Durchschnittsvektor aller bekannten Wörter im Urteil. """
    words = [w for w in str(doc).split() if w in model.wv]
    if not words:
        # Falls kein Wort des Urteils im Training vorkam, geben wir einen Null-Vektor zurück
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

# 2. Vektorisierung der Trainings- und Testdaten
# WICHTIG: Wir nutzen das w2v_model aus Schritt 5.2 (nur auf Train trainiert!)
print("Vektorisierung der Trainingsdaten...")
X_train_vec = np.array([get_doc_vector(text, w2v_model) for text in df_lg_train["cleaned_text"]])

# Hinweis: Wir bereiten hier auch die Testdaten vor, damit wir sie später evaluieren können.
# Falls df_lg_test noch nicht bereinigt wurde, holen wir das hier kurz nach:
if "cleaned_text" not in df_lg_test.columns:
    print("Bereinige Testdaten für die Evaluation...")
    df_lg_test["cleaned_text"] = df_lg_test["text"].apply(legal_preprocess)

print("Vektorisierung der Testdaten (mit Trainings-Modell)...")
X_test_vec = np.array([get_doc_vector(text, w2v_model) for text in df_lg_test["cleaned_text"]])

# 3. Auswahl der strukturierten Features (aus der Gemini-Extraktion in Abschnitt 4)
# Wir füllen fehlende Werte (NaN) mit 0, damit die ML-Modelle damit arbeiten können.
struct_cols = ['Kaufpreis_num']
X_train_structured = df_lg_train[struct_cols].fillna(0)
X_test_structured = df_lg_test[struct_cols].fillna(0)

# 4. Umwandlung der Word2Vec-Vektoren in DataFrames
emb_cols = [f"emb_{i}" for i in range(X_train_vec.shape[1])]
df_train_features = pd.DataFrame(X_train_vec, columns=emb_cols)
df_test_features = pd.DataFrame(X_test_vec, columns=emb_cols)

# 5. Finale Zusammenführung (Struktur + Text-Embeddings)
# WICHTIG: reset_index(drop=True) sorgt dafür, dass die Zeilen beim Zusammenfügen exakt matchen!
X_train_final = pd.concat([X_train_structured.reset_index(drop=True), df_train_features], axis=1)
X_test_final = pd.concat([X_test_structured.reset_index(drop=True), df_test_features], axis=1)

print("-" * 30)
print(f"✅ Feature-Matrizen erfolgreich erstellt.")
print(f"📊 Training: {X_train_final.shape[0]} Urteile, {X_train_final.shape[1]} Features")
print(f"📊 Test:     {X_test_final.shape[0]} Urteile, {X_test_final.shape[1]} Features")

## 6. Analyse und Auswertung

Entscheidungsbaum

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, accuracy_score

# 1. Modell-Initialisierung
dt_model = DecisionTreeClassifier(max_depth=10, random_state=42, class_weight="balanced")

# 2. Cross-Validation (auf Trainingsdaten)
# Wir prüfen die Stabilität über 5 Folds
cv_results_dt = cross_validate(
    dt_model, X_train_final, y_train, 
    cv=cv, # Das cv-Objekt (StratifiedKFold) aus Abschnitt 4.4
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    return_train_score=False
)

print(f"--- Cross-Validation (Train) ---")
print(f"Mittlere Accuracy: {cv_results_dt['test_accuracy'].mean():.4f} (+/- {cv_results_dt['test_accuracy'].std():.4f})")
print(f"Mittlerer Recall:   {cv_results_dt['test_recall'].mean():.4f}")

# 3. Finales Training & Test-Evaluation
dt_model.fit(X_train_final, y_train)
y_pred_dt = dt_model.predict(X_test_final)

print(f"\n--- Finale Evaluation (Testset) ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(classification_report(y_test, y_pred_dt))

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Modell-Initialisierung
rf_model = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, class_weight="balanced")

# 2. Cross-Validation (auf Trainingsdaten)
cv_results_rf = cross_validate(
    rf_model, X_train_final, y_train, 
    cv=cv, 
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    return_train_score=False
)

print(f"--- Cross-Validation (Train) ---")
print(f"Mittlere Accuracy: {cv_results_rf['test_accuracy'].mean():.4f} (+/- {cv_results_rf['test_accuracy'].std():.4f})")

# 3. Finales Training & Test-Evaluation
rf_model.fit(X_train_final, y_train)
y_pred_rf = rf_model.predict(X_test_final)

print(f"\n--- Finale Evaluation (Testset) ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(classification_report(y_test, y_pred_rf))

Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import pandas as pd

# 1. Modell-Initialisierung
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# 2. Cross-Validation (auf Trainingsdaten)
cv_results_gb = cross_validate(
    gb_model, X_train_final, y_train, 
    cv=cv, 
    scoring=['accuracy', 'precision', 'recall', 'f1'],
    return_train_score=False
)

print(f"--- Cross-Validation (Train) ---")
print(f"Mittlere Accuracy: {cv_results_gb['test_accuracy'].mean():.4f}")

# 3. Finales Training & Test-Evaluation
gb_model.fit(X_train_final, y_train)
y_pred_gb = gb_model.predict(X_test_final)

print(f"\n--- Finale Evaluation (Testset) ---")
print(classification_report(y_test, y_pred_gb))

# 4. Visualisierung der Feature Importance
# Wir zeigen die Top 15 Merkmale (Strukturvariablen + Embeddings)
importances = pd.Series(gb_model.feature_importances_, index=X_train_final.columns)
plt.figure(figsize=(10, 6))
importances.nlargest(15).sort_values().plot(kind='barh', color='skyblue')
plt.title("Top 15 Features (Gradient Boosting)")
plt.xlabel("Gini Importance")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

evtl. SHAP Werte für Erklärbarkeit